In [1]:
import requests
import json
import pandas as pd
import os
from tqdm.notebook import tqdm  # Barra de progreso para notebooks

Función para Obtener Datos de Clientes

In [2]:
def get_client_data(client_id):
    url = 'https://faas-lon1-917a94a7.doserverless.co/api/v1/web/' \
          'fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/clients-data'
    params = {'client_id': client_id}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        return data.get('values', {})
    except requests.exceptions.RequestException as e:
        print(f'Error al obtener datos del cliente {client_id}: {e}')
        return None

Función para Obtener Datos de Tarjetas

In [3]:
def get_card_data(client_id):
    url = 'https://faas-lon1-917a94a7.doserverless.co/api/v1/web/' \
          'fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/cards-data'
    params = {'client_id': client_id}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        return data.get('values', {})
    except requests.exceptions.RequestException as e:
        print(f'Error al obtener datos de tarjetas para el cliente {client_id}: {e}')
        return None

In [4]:
transactions = pd.read_csv('data/raw/transactions_data.csv', parse_dates=['date'])

In [5]:
client_ids = transactions['client_id'].unique()
print(f'Se encontraron {len(client_ids)} clientes únicos.')

Se encontraron 1219 clientes únicos.


In [6]:
os.makedirs('data/processed', exist_ok=True)

In [7]:
all_client_data = {}

for client_id in tqdm(client_ids, desc='Descargando datos de clientes'):
    client_data = get_client_data(client_id)
    if client_data is not None:
        all_client_data[str(client_id)] = client_data  # Convertir client_id a str

Descargando datos de clientes:   0%|          | 0/1219 [00:00<?, ?it/s]

In [8]:
with open('data/processed/clients_data.json', 'w') as f:
    json.dump(all_client_data, f)

In [ ]:
# Descargar y almacenar los datos de tarjetas
all_card_data = {}

for client_id in tqdm(client_ids, desc='Descargando datos de tarjetas'):
    card_data = get_card_data(client_id)
    if card_data is not None:
        all_card_data[str(client_id)] = card_data  # client_id es str

In [11]:
# Guardar los datos en un archivo JSON
with open('data/processed/cards_data.json', 'w') as f:
    json.dump(all_card_data, f)

Descargar Archivo train_fraud_labels.json

In [ ]:
import requests

url = 'https://cdn.nuwe.io/challenges-ds-datasets/hackathon-caixabank-data-24/train_fraud_labels.json'
response = requests.get(url)

if response.status_code == 200:
    with open('data/raw/train_fraud_labels.json', 'wb') as f:
        f.write(response.content)
    print('Archivo train_fraud_labels.json descargado correctamente.')
else:
    print('Error al descargar el archivo train_fraud_labels.json.')